In [0]:
CREATE OR REPLACE TEMP VIEW exploded_directors AS
SELECT 
  movie_id,  
  trim(director_element) AS director
FROM 
  hive_metastore.default.dim_movie
LATERAL VIEW explode(split(director, ",")) AS director_element;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.dim_director AS
SELECT 
  ROW_NUMBER() OVER (ORDER BY director) AS director_id,  
  TRIM(director) AS director  
FROM (
  SELECT DISTINCT TRIM(director) AS director
  FROM exploded_directors
) t;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.bridge_movie_director AS
SELECT DISTINCT 
  ed.movie_id,  
  dd.director_id
FROM exploded_directors ed
JOIN hive_metastore.default.dim_director dd  
  ON TRIM(ed.director) = dd.director;

In [0]:
SELECT * 
FROM hive_metastore.default.bridge_movie_director 
LIMIT 10;